In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
import os
import logging

logging.basicConfig(level=logging.INFO)

NUM_OF_YEARS = 10

# Функција за проверка дали стрингот содржи бројка
def has_num(shifra):
    return any(char.isdigit() for char in shifra)

# Функција за преземање на тикерите
def fetch_tikeri_bs():
    url = 'https://www.mse.mk/mk/stats/symbolhistory/kmb'
    response = requests.get(url)
    
    if response.status_code != 200:
        logging.error("Не успеав да ја превземам страната.")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    dropdown = soup.find('select', {'id': 'Code'})
    options = dropdown.find_all('option') if dropdown else []
    
    # Преземи тикери без броеви
    tikeri = [option['value'] for option in options if not has_num(option['value'])]
    return tikeri

# Функција за преземање на историски податоци за секој тикер
def fetch_historic_data_bs4(ticker):
    base_url = f"https://www.mse.mk/mk/stats/symbolhistory/{ticker.lower()}"
    historic_data = []
    date_to = datetime.now()
    
    for _ in range(NUM_OF_YEARS):
        date_from = date_to - timedelta(days=365)
        params = {
            "FromDate": date_from.strftime("%d.%m.%Y"),
            "ToDate": date_to.strftime("%d.%m.%Y"),
        }
        
        try:
            response = requests.get(base_url, params=params)
            if response.status_code != 200:
                logging.warning(f"Неуспешно преземање на податоци за {ticker}.")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.select_one('#resultsTable > tbody')

            if table:
                rows = table.find_all('tr')
                for row in rows:
                    data_row = [cell.text.strip() for cell in row.find_all('td')]
                    if data_row:
                        # Додај го тикерот како прва колона
                        data_row.insert(0, ticker)
                        historic_data.append(data_row)
            
            date_to = date_from

        except Exception as e:
            logging.error(f"Грешка при преземање на податоци за {ticker}: {e}")
    
    return historic_data

# Функција за зачувување на податоци во заеднички CSV фајл
def save_to_csv(data):
    filename = "mse_data.csv"
    file_exists = os.path.isfile(filename)
    
    # Отвори CSV фајл во режим на додавање (append)
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Додај ја заглавната линија ако фајлот е нов
        if not file_exists:
            writer.writerow(["Tiker", "Datum", "Otvorena cena", "Najgolema cena", "Najmala cena", "Zatvorena cena", "Volumen", "Promet"])
        
        # Запиши ги податоците
        for row in data:
            writer.writerow(row)

# Главна функција за преземање и зачувување на податоци за сите тикери
def main():
    tickers = fetch_tikeri_bs()
    logging.info(f"Пронајдени тикери: {tickers}")

    all_data = []
    
    for ticker in tickers:
        logging.info(f"Преземам податоци за {ticker}...")
        data = fetch_historic_data_bs4(ticker)
        
        if data:
            all_data.extend(data)
        else:
            logging.warning(f"Нема достапни податоци за {ticker}.")
    
    if all_data:
        save_to_csv(all_data)
        logging.info("Сите податоци се зачувани во mse_data.csv.")
    else:
        logging.warning("Нема нови податоци за зачувување.")

# Повикај ја главната функција
if __name__ == "__main__":
    main()

INFO:root:Пронајдени тикери: ['ADIN', 'ALK', 'ALKB', 'AMEH', 'APTK', 'ATPP', 'AUMK', 'BANA', 'BGOR', 'BIKF', 'BIM', 'BLTU', 'CBNG', 'CDHV', 'CEVI', 'CKB', 'CKBKO', 'DEBA', 'DIMI', 'EDST', 'ELMA', 'ELNC', 'ENER', 'ENSA', 'EUHA', 'EUMK', 'EVRO', 'FAKM', 'FERS', 'FKTL', 'FROT', 'FUBT', 'GALE', 'GDKM', 'GECK', 'GECT', 'GIMS', 'GRDN', 'GRNT', 'GRSN', 'GRZD', 'GTC', 'GTRG', 'IJUG', 'INB', 'INHO', 'INOV', 'INPR', 'INTP', 'JAKO', 'JUSK', 'KARO', 'KDFO', 'KJUBI', 'KKST', 'KLST', 'KMB', 'KMPR', 'KOMU', 'KONF', 'KONZ', 'KORZ', 'KPSS', 'KULT', 'KVAS', 'LAJO', 'LHND', 'LOTO', 'LOZP', 'MAGP', 'MAKP', 'MAKS', 'MB', 'MERM', 'MKSD', 'MLKR', 'MODA', 'MPOL', 'MPT', 'MPTE', 'MTUR', 'MZHE', 'MZPU', 'NEME', 'NOSK', 'OBPP', 'OILK', 'OKTA', 'OMOS', 'OPFO', 'OPTK', 'ORAN', 'OSPO', 'OTEK', 'PELK', 'PGGV', 'PKB', 'POPK', 'PPIV', 'PROD', 'PROT', 'PTRS', 'RADE', 'REPL', 'RIMI', 'RINS', 'RZEK', 'RZIT', 'RZIZ', 'RZLE', 'RZLV', 'RZTK', 'RZUG', 'RZUS', 'SBT', 'SDOM', 'SIL', 'SKON', 'SKP', 'SLAV', 'SNBT', 'SNBTO', 'SOL